In [617]:
import pandas as pd
import sys
reload(sys)
sys.setdefaultencoding('utf8')


In [618]:
df = pd.read_csv("../training_ironita2018.tsv", sep='\t')
df.head(3)


,id,text,irony,sarcasm,topic
0,811156813181841408,"Zurigo, trovato morto il presunto autore della sparatoria nel centro islamico #20dicembre https://t.co/rBjvUi8RJ2",0,0,HSC
1,811183087350595584,"Zurigo, trovato morto il presunto autore della sparatoria nel centro islamico - https://t.co/XMrEqPnV6w tramite https://t.co/zAAl3RtO5F",0,0,HSC
2,826380632376881152,"Zingari..i soliti ""MERDOSI""..#cacciamolivia Roma, i rom aggrediscono un 81enne per rapinarlo. Bloccati dai cittadini https://t.co/j6LYylIkhe",0,0,HSC


In [619]:
df.info()

In [620]:
pd.options.display.max_colwidth = 100
df[(df.irony==0) & (df.sarcasm ==1)].head(3)

,id,text,irony,sarcasm,topic


In [621]:
pd.options.display.max_colwidth = 400
df[(df.irony==1) | (df.sarcasm ==1)][['text','irony','sarcasm']].sample(20)

,text,irony,sarcasm
2668,#Risorse da accogliere... Omicidi e stupri nel campo #profughi in #Libia: arrestato 22enne somalo a #Milano. https://t.co/Y2CO7y0UbU,1,1
117,Uau mario monti ci fa il regalo dinatale e approda su twitter. Che culo.,1,0
2236,Chi è entrato nel cellulare di Jennifer Lawrence potrebbe entrare anche nel tuo. E troverebbe sempre le foto della Lawrence. [@TheAubergine],1,0
3878,"“la felicità non ha prezzo“, la mia sì, costa circa 3,50€ e si chiama Nutella. #nonpuoimancare",1,0
1974,"Ebola, il medico italiano guarisce grazie a una cura sperimentale. Che ora verrà testata sui vigili di Roma. [@ZipSatiraLampo]",1,0
324,"Se qualcuno di voi mi dimostra che negri, rom, islamici e clandestini sono davvero risorse, GIURO CHE BACIO IL CULO ALLA KYENGE!!!",1,1
3637,@GaspErCarbonaro #labuonascuola del Gelataio è anche questa meglio che imparino da subito. A tutti in omaggio Oliver Twist...,1,0
2078,"delocalizziamo alla Libia la gestione dei migranti, ottimo https://t.co/8ZYXQy9a9l",1,1
3662,"@forummediaset dite alla Contessa che i rom hanno macchinoni e oro a cianfe perchè LI RUBANO, non certo perché lavorano onestAmente! #schifo",1,1
787,Ora che sono stati scoperti nuovi pianeti potreste anche mandarci i migranti. Sarebbe un mondo tutto per loro.,1,1


In [622]:
df[(df.irony==1) | (df.sarcasm ==1)].count()

id         2023
text       2023
irony      2023
sarcasm    2023
topic      2023
dtype: int64

In [623]:
'''
bashCommand = "java -version"
import subprocess
process = subprocess.Popen(['/bin/sh', 'java', '-version'], stdout=subprocess.PIPE)
output, error = process.communicate()
print(output)
print(error)

subprocess.Popen(['/bin/sh', 'java', '-version'])
from subprocess import check_output
out = check_output(["java", "-version"])
print(out)

'''

'\nbashCommand = "java -version"\nimport subprocess\nprocess = subprocess.Popen([\'/bin/sh\', \'java\', \'-version\'], stdout=subprocess.PIPE)\noutput, error = process.communicate()\nprint(output)\nprint(error)\n\nsubprocess.Popen([\'/bin/sh\', \'java\', \'-version\'])\nfrom subprocess import check_output\nout = check_output(["java", "-version"])\nprint(out)\n\n'

In [624]:
def generaArraydataset(df=df, target="irony", lower = True):
    arrayDataset = []
    arrayTarget = []
    for index, row in df.iterrows():
        if(lower):
            arrayDataset.append(row["text"].lower())
        else:
            arrayDataset.append(row["text"])
        if(row[target] == 0):
            arrayTarget.append(-1)
        else:
            arrayTarget.append(row[target])
    return [arrayDataset, arrayTarget]

array = generaArraydataset()
X_train = array[0]
y_train_irony = array[1]
y_train_sarcasm = generaArraydataset(target="sarcasm")[1]

'''
from nltk import word_tokenize
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer(strip_handles=True)

#token = nltk.word_tokenize(X_train[2])
print(tknzr.tokenize(X_train[2]))
'''
print(y_train_irony[2])
print(y_train_sarcasm[2])

In [625]:
from nltk import word_tokenize
from nltk.tokenize import TweetTokenizer
import hashlib

def creaBoW(dataset, target, name = "dataset.klp"):
    f = open(name,"w+")
    for index, riga in enumerate(dataset):
        string = str(target[index]) + " |BV:bow| "
        tknzr = TweetTokenizer(strip_handles=True)
        tweettoknz = tknzr.tokenize(dataset[index])
        for word in tweettoknz:
            string = string + "_" + hashlib.md5(''.join(word)).hexdigest() + ":1.0 "
        f.write(string + "|EV| |BS:comment| "+ dataset[index] +" |ES| |BS:IDTweet| "+ str(df.iloc[index]['id']) +" |ES| \n")    
    f.close()
    
NOME_FILE = "dataset_ironia.klp"
#creaBoW(X_train, y_train_sarcasm, NOME_FILE)
creaBoW(X_train, y_train_irony, NOME_FILE)

In [626]:
df = pd.read_csv("../training_ironita2018_renlt_processed.tsv", sep='\t')
df.head(3)


,id,text,irony,sarcasm,topic,text::text::S
0,811156813181841408,"Zurigo, trovato morto il presunto autore della sparatoria nel centro islamico #20dicembre https://t.co/rBjvUi8RJ2",0,0,HSC,"Zurigo::zurigo::SP ,::,::FF trovato::trovare::V morto::morire::V il::il::RD presunto::presumere::V autore::autore::S della::di::EA sparatoria::sparatoria::S nel::in::EA centro::centro::S islamico::islamico::A #20dicembre::#20dicembre::HTG LINK::link::$URL"
1,811183087350595584,"Zurigo, trovato morto il presunto autore della sparatoria nel centro islamico - https://t.co/XMrEqPnV6w tramite https://t.co/zAAl3RtO5F",0,0,HSC,"Zurigo::zurigo::SP ,::,::FF trovato::trovare::V morto::morire::V il::il::RD presunto::presumere::V autore::autore::S della::di::EA sparatoria::sparatoria::S nel::in::EA centro::centro::S islamico::islamico::A -::-::FC LINK::link::$URL tramite::tramite::S LINK::link::$URL"
2,826380632376881152,"Zingari..i soliti ""MERDOSI""..#cacciamolivia Roma, i rom aggrediscono un 81enne per rapinarlo. Bloccati dai cittadini https://t.co/j6LYylIkhe",0,0,HSC,"Zingari::zingari::SP .::.::FF i::i::RD soliti::solito::S ""::""::FB MERDOSI::merdosi::SP ""::""::FB .::.::FF #cacciamolivia::#cacciamolivia::HTG Roma::roma::S ,::,::FF i::i::RD rom::rom::S aggrediscono::aggredire::V un::un::RI 81::81::N enne::enne::S per::per::E rapinarlo::rapinarlo::V .::.::FF Bloccati::bloccare::V dai::da::EA cittadini::cittadino::S LINK::link::$URL"


In [627]:
def generaArrayTokenizzatoLemmi(df=df):
    arrayLemmi = []
    for index, row in df.iterrows():
        arrayLemmi.append(row["text::text::S "])
    arrayTokenizzatoLemmi = []
    #per ogni elemento del dataset
    for text in arrayLemmi:
        arrayParoleLemm = text.split()
        arrayFrase = []
        #per ogni token text::text::S
        for parola in arrayParoleLemm:
            arrayParola = parola.split("::")
            #divido ogni singolo token e prendo il lemma
            arrayFrase.append(arrayParola[1])
        arrayTokenizzatoLemmi.append(arrayFrase)
    return arrayTokenizzatoLemmi

def generaArrayTokenizzatoSurface(df=df, listPOS = ['A','B','S','V','HTG'], lower = True):
    arrayLemmi = []
    for index, row in df.iterrows():
        arrayLemmi.append(row["text::text::S "])
    arrayTokenizzatoLemmi = []
    for text in arrayLemmi:
        arrayParoleLemm = text.split()
        arrayFrase = []
        for parola in arrayParoleLemm:
            arrayParola = parola.split("::")
            if((arrayParola[2] in listPOS) and listPOS != []):
                if(lower):
                    arrayFrase.append(arrayParola[0].lower())
                else:
                    arrayFrase.append(arrayParola[0])
            if(listPOS == []):
                if(lower):
                    arrayFrase.append(arrayParola[0].lower())
                else:
                    arrayFrase.append(arrayParola[0])
        arrayTokenizzatoLemmi.append(arrayFrase)
    return arrayTokenizzatoLemmi
    
prova = generaArrayTokenizzatoLemmi()
print prova[2]
prova2 = generaArrayTokenizzatoSurface()
print prova2[2]
arrayCompleto = generaArrayTokenizzatoSurface(listPOS = [])
print arrayCompleto[2]

In [628]:
import nltk
from nltk.util import ngrams

def generaArrayNgrams(dataset, ngram=2):
    arrayBigrammi = []
    for item in dataset:
        arrayBigrammi.append(list(ngrams(item,ngram)))
    return arrayBigrammi


arrayBigrammiLemmi = generaArrayNgrams(prova)
arrayBigrammiSurface = generaArrayNgrams(arrayCompleto)


arrayBigrammiChar = generaArrayNgrams(X_train)
array3grammiChar = generaArrayNgrams(X_train, 3)
array4grammiChar = generaArrayNgrams(X_train, 4)
array5grammiChar = generaArrayNgrams(X_train, 5)

In [629]:
import numpy as np
from numpy import asarray

def importaWordSpace(pathWS = "../ws"):
    # load the whole embedding into memory
    embeddings_index = dict()
    f = open(pathWS)
    for line in f:
        values = line.split()
        word = values[0]
        #print(word)
        string = ''.join(values[3:]).split(',')
        for x in string:
            x = float(x)
        #print(string)
        coefs = asarray(string, dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('Loaded %s word vectors.' % len(embeddings_index))
    return embeddings_index

def importaWordSpace2(pathWS = '../DPL-IT_lrec2016.txt'):
    # load the whole embedding into memory
    embeddings_index = dict()
    f = open(pathWS)
    for line in f:
        values = line.split()
        word = values[0]
        #print(word)
        string = ''.join(values[1:]).split(',')
        for x in string:
            x = float(x)
        #print(string)
        coefs = asarray(string, dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('Loaded %s word vectors.' % len(embeddings_index))
    return embeddings_index

def importaWordSpace3(pathWS = '../emoji2vec.txt'):
    # load the whole embedding into memory
    embeddings_index = dict()
    f = open(pathWS)
    for line in f:
        values = line.split()
        word = values[0]
        coefs = asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('Loaded %s word vectors.' % len(embeddings_index))
    return embeddings_index

def generaCombWordSpace(dataset = prova2):
    embeddings_index = importaWordSpace()
    combDataset = []
    for tweet in dataset:
        comb = np.zeros(250)
        norm = 0.0
        for parola in tweet:
            if(parola in embeddings_index):
                comb = comb + embeddings_index[parola]
                norm += 1
        if(norm != 0):
            combDataset.append(comb/norm)
        else:
            combDataset.append(comb)
    return combDataset

'''
def getCentroide(dataset = prova2):
    embeddings_index = importaWordSpace()
    combDataset = []
    for tweet in dataset:
        comb = np.zeros(250)
        i = 0
        for parola in tweet:
            if(parola in embeddings_index):
                comb = comb + embeddings_index[parola]
                i = i + 1
        if(i == 0):
            i = 1
        combDataset.append(comb/i)
    return combDataset
'''
def generaCombDPL(dataset = prova2):
    embeddings_index = importaWordSpace2()
    combDataset = []
    for tweet in dataset:
        comb = np.zeros(3)
        norm = 0.0
        for parola in tweet:
            if(parola in embeddings_index):
                comb = comb + embeddings_index[parola]
                norm += 1
        if(norm == 0):
            norm = 1
        combDataset.append(comb/norm)
    return combDataset

                

In [630]:
from nltk import word_tokenize
from nltk.tokenize import TweetTokenizer
import numpy as np

def generaEmoji(dataset = X_train):
    embeddings_index = importaWordSpace3()
    combDataset = []
    for tweet in dataset:
        tknzr = TweetTokenizer(strip_handles=True)
        tweettoknz = tknzr.tokenize(tweet)
        comb = np.zeros(300)
        for parola in tweettoknz:
            if(parola in embeddings_index):
                print parola
                comb = comb + embeddings_index[parola]
        combDataset.append(comb)
    return combDataset

def feturesPunteggiatura(dataset = X_train):
    combDataset = []
    for tweet in dataset:
        charset = [0.0,0.0,0.0,0.0,0.0,0.0]
        numUpperCase = 0.0
        numchar = 0.0
        for char in tweet:
            numchar += 1
            if(char.istitle()):
                numUpperCase += 1
            if(char == "!"):
                charset[0] += 1
            if(char == "?"):
                charset[1] += 1
            if(char == "."):
                charset[2] += 1    
            if(char == ","):
                charset[3] += 1
            if(char == ";"):
                charset[4] += 1
            if(char == ":"):
                charset[5] += 1
        charset.append(numUpperCase/numchar)
        combDataset.append(charset)
    return combDataset 

In [631]:
from scipy.spatial import distance

#Combinazione Lineare delle parole del Word Space moltiplicate per la distanza dal centroide
#Per ogni Elemento del dataset ritorna la tupla (Bow, Var, Mean)
def generaIronySpecificBow(dataset, arrayCentroide = generaCombWordSpace(dataset=prova)):
    embeddings_index = importaWordSpace()
    combDataset = []
    for index, tweet in enumerate(dataset):
        comb = np.zeros(250)
        arrayDistanze = []
        for parola in tweet:
            if(parola in embeddings_index):
                comb = comb + embeddings_index[parola] * distance.cosine(embeddings_index[parola], arrayCentroide[index])
                arrayDistanze.append(distance.euclidean(embeddings_index[parola], arrayCentroide[index]))
        x = asarray(arrayDistanze)
        aList = (comb, np.var(x), np.mean(x))
        combDataset.append(tuple(aList))
    return combDataset

In [632]:
from scipy.spatial import distance

def appendBowIronySpecific(dataset, nomerappresentazione, file_name = "dataset.klp", embeddings_index = importaWordSpace(), arrayCentroide = generaCombWordSpace(dataset=prova)):
    f = open(file_name, 'r')
    file_lines = []
    combDataset = []
    for index, x in enumerate(f.readlines()):
        string = " |BV:"+ nomerappresentazione + "| "
        arrayDistanze = []
        for word in dataset[index]:
            #calcola distanza tra word e centroide[index]
            if(word in embeddings_index):
                distanza = distance.cosine(arrayCentroide[index], embeddings_index[word])
                arrayDistanze.append(distance.cosine(arrayCentroide[index], embeddings_index[word]))
            else:
                distanza = 1.0
            string = string + "_" + hashlib.md5(''.join(word)).hexdigest() + ":"+ str(distanza)+" "    
        string = string + "|EV| "
        file_lines.append(''.join([x.strip(), string, '\n']))
        x = asarray(arrayDistanze)
        if(np.isnan(np.var(x)) or np.isnan(np.mean(x))):
            aList = [0,0]
        else:
            aList = [np.var(x), np.mean(x)]
        combDataset.append(aList)

    with open(file_name, 'w') as f:
        f.writelines(file_lines)
    
    return combDataset

In [633]:
import hashlib


def appendBow(dataset, nomerappresentazione, file_name = "dataset.klp", tupla = True):
    f = open(file_name, 'r')
    file_lines = []
    for index, x in enumerate(f.readlines()):
        string = " |BV:"+ nomerappresentazione + "| "
        for word in dataset[index]:
            if(tupla):
                string = string + "_" + hashlib.md5(''.join(word)).hexdigest() + ":1.0 "
            else:
                string = string + "_" + word + ":1.0 "
        string = string + "|EV| "
        file_lines.append(''.join([x.strip(), string, '\n']))

    with open(file_name, 'w') as f:
        f.writelines(file_lines) 

def appendDenseRapr(dataset, nomerappresentazione, file_name = "dataset.klp"):
    f = open(file_name, 'r')
    file_lines = []
    for index, x in enumerate(f.readlines()):
        string = " |BDV:"+ nomerappresentazione + "| "
        for num in dataset[index]:
            string = string + str(num) + " "
        string = string + "|EDV| "
        file_lines.append(''.join([x.strip(), string, '\n']))
        
    with open(file_name, 'w') as f:
        f.writelines(file_lines)


def appendDenseRaprTuple(dataset, nomerappresentazione, idTupla, file_name = "dataset.klp"):
    f = open(file_name, 'r')
    file_lines = []
    for index, x in enumerate(f.readlines()):
        string = " |BDV:"+ nomerappresentazione + "| "
        if( idTupla < 1):
            for num in dataset[index][idTupla]:
                string = string + str(num) + " "
        else:
            string = string + str(dataset[index][idTupla]) + " "
        string = string + "|EDV| "
        file_lines.append(''.join([x.strip(), string, '\n']))
        
    with open(file_name, 'w') as f:
        f.writelines(file_lines)
        

In [634]:
combWS = generaCombWordSpace()
appendDenseRapr(combWS, "WSSurface",NOME_FILE)



comDPL = generaCombDPL()
appendDenseRapr(comDPL, "combDPL",NOME_FILE)

combEmoji = generaEmoji()
appendDenseRapr(combEmoji, "combEmoji",NOME_FILE)

featuresPuntegg = feturesPunteggiatura()
appendDenseRapr(featuresPuntegg, "featPunt",NOME_FILE)

In [635]:
meanVarArray = appendBowIronySpecific(prova, "bowIronySpecific", NOME_FILE)
appendDenseRapr(meanVarArray, "VarMean",NOME_FILE)
IronySpecificASV = generaArrayTokenizzatoSurface(listPOS = ['A','S','V'])
meanVarArray2 = appendBowIronySpecific(IronySpecificASV, "bowIronySpecificASV", NOME_FILE)
appendDenseRapr(meanVarArray2, "VarMeanASV",NOME_FILE)

In [636]:

irony_specific_bow = generaIronySpecificBow(arrayCompleto)
appendDenseRaprTuple(irony_specific_bow, "IronySpecificWS", 0 ,NOME_FILE)
#appendDenseRaprTuple(irony_specific_bow, "Var", 1 ,NOME_FILE)
#appendDenseRaprTuple(irony_specific_bow, "Mean", 2 ,NOME_FILE)


In [637]:
appendBow(prova, "bowLemmi", NOME_FILE, tupla = False)
appendBow(arrayBigrammiLemmi, "bowBigramLemmi", NOME_FILE)
appendBow(arrayBigrammiSurface, "bowBigramSurface", NOME_FILE)



appendBow(arrayBigrammiChar, "bow2gramChar", NOME_FILE)
appendBow(array3grammiChar, "bow3gramChar", NOME_FILE)
appendBow(array4grammiChar, "bow4gramChar", NOME_FILE)
appendBow(array5grammiChar, "bow5gramChar", NOME_FILE)